In [1]:
import requests
import shutil
import urllib
import json
from datetime import datetime
import os
import pandas as pd
import io
import re
import csv
#New code replace URL

In [2]:
burl_url = 'https://services1.arcgis.com/X3dmaNvzWUnemDAc/arcgis/rest/services/Cases_current/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=name%20asc&resultOffset=0&resultRecordCount=50&resultType=standard&cacheHint=true'

In [3]:
requests.get(burl_url)

<Response [200]>

In [4]:
burl_response = requests.get(burl_url)

In [5]:
burl_response.content

b'{"objectIdFieldName":"ObjectId","uniqueIdField":{"name":"ObjectId","isSystemMaintained":true},"globalIdFieldName":"GlobalID","geometryType":"esriGeometryPolygon","spatialReference":{"wkid":102100,"latestWkid":3857},"fields":[{"name":"name","type":"esriFieldTypeString","alias":"Name","sqlType":"sqlTypeOther","length":1000,"domain":null,"defaultValue":null},{"name":"GlobalID","type":"esriFieldTypeGlobalID","alias":"GlobalID","sqlType":"sqlTypeOther","length":38,"domain":null,"defaultValue":null},{"name":"CreationDate","type":"esriFieldTypeDate","alias":"CreationDate","sqlType":"sqlTypeOther","length":8,"domain":null,"defaultValue":null},{"name":"Creator","type":"esriFieldTypeString","alias":"Creator","sqlType":"sqlTypeOther","length":128,"domain":null,"defaultValue":null},{"name":"EditDate","type":"esriFieldTypeDate","alias":"EditDate","sqlType":"sqlTypeOther","length":8,"domain":null,"defaultValue":null},{"name":"Editor","type":"esriFieldTypeString","alias":"Editor","sqlType":"sqlType

In [6]:
burl_parsed_json = json.loads(burl_response.content)

In [7]:
burl_parsed_json

{'objectIdFieldName': 'ObjectId',
 'uniqueIdField': {'name': 'ObjectId', 'isSystemMaintained': True},
 'globalIdFieldName': 'GlobalID',
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'fields': [{'name': 'name',
   'type': 'esriFieldTypeString',
   'alias': 'Name',
   'sqlType': 'sqlTypeOther',
   'length': 1000,
   'domain': None,
   'defaultValue': None},
  {'name': 'GlobalID',
   'type': 'esriFieldTypeGlobalID',
   'alias': 'GlobalID',
   'sqlType': 'sqlTypeOther',
   'length': 38,
   'domain': None,
   'defaultValue': None},
  {'name': 'CreationDate',
   'type': 'esriFieldTypeDate',
   'alias': 'CreationDate',
   'sqlType': 'sqlTypeOther',
   'length': 8,
   'domain': None,
   'defaultValue': None},
  {'name': 'Creator',
   'type': 'esriFieldTypeString',
   'alias': 'Creator',
   'sqlType': 'sqlTypeOther',
   'length': 128,
   'domain': None,
   'defaultValue': None},
  {'name': 'EditDate',
   'type': 'esriFieldTypeDate',
   'alia

In [8]:
burl_parsed_json['features']

[{'attributes': {'name': 'BASS RIVER TWP',
   'GlobalID': '91220d12-20b8-491c-9a7e-603394913b6d',
   'CreationDate': 1585695656556,
   'Creator': 'merrilee',
   'EditDate': 1585695656556,
   'Editor': 'merrilee',
   'Shape__Area': 342864210.988281,
   'Shape__Length': 104639.482017116,
   'name_1584474306361': 'BASS RIVER TWP',
   'reportdt': 1591759860000,
   'confirmed': 7,
   'recovered': 0,
   'deaths': 0,
   'active': 7,
   'GlobalID_1584474306361': '9971a577-71e3-44e3-85a9-e9a9fb2094f5',
   'tested': 7,
   'CreationDate_1584474306361': 1591759974676,
   'Creator_1584474306361': 'merrilee',
   'EditDate_1584474306361': 1591762795454,
   'Editor_1584474306361': 'merrilee',
   'ObjectId': 1}},
 {'attributes': {'name': 'BEVERLY CITY',
   'GlobalID': '01c27639-9af8-49e1-b843-9e922fd8047f',
   'CreationDate': 1585695656556,
   'Creator': 'merrilee',
   'EditDate': 1585695656556,
   'Editor': 'merrilee',
   'Shape__Area': 3365427.26953125,
   'Shape__Length': 7207.80643191792,
   'name_

In [9]:
burl_content = ''
#vim xed visualcode
#pycharm
hist_df = pd.read_csv('Webscraping_Db.csv')
for city in burl_parsed_json['features']:
    cityname = (city['attributes']['name'])
    city_df = hist_df[hist_df['City'] == cityname]
    cur_data = int(city['attributes']['confirmed'])
    current = max(max(city_df['Total_Cases']), cur_data) #if Max(total)>max(confirmed) then "Call"
    county = city_df['County'].values[0]
    state = city_df['State'].values[0]
    date = datetime.fromtimestamp(int(str(city['attributes']['reportdt'])[:-3]))
    burl_content += f'{state}, {county}, {cityname}, {current}, {date.date().isoformat()}, {not (cur_data != current)}\n'
    #cur_data is different than maximum of both current and cur_data.  
    #True means that values are correct.  False means that values are incorrect and cannot fix them
    
#print(city_df.head())
print(burl_content)

#

New Jersey, Burlington, BASS RIVER TWP, 7, 2020-06-09, True
New Jersey, Burlington, BEVERLY CITY, 26, 2020-06-09, True
New Jersey, Burlington, BORDENTOWN CITY, 30, 2020-06-09, True
New Jersey, Burlington, BORDENTOWN TWP, 77, 2020-06-09, True
New Jersey, Burlington, BURLINGTON CITY, 97, 2020-06-09, True
New Jersey, Burlington, BURLINGTON TWP, 432, 2020-06-09, True
New Jersey, Burlington, CHESTERFIELD TWP, 127, 2020-06-09, True
New Jersey, Burlington, CINNAMINSON TWP, 154, 2020-06-09, True
New Jersey, Burlington, DELANCO TWP, 41, 2020-06-09, False
New Jersey, Burlington, DELRAN TWP, 143, 2020-06-09, True
New Jersey, Burlington, EASTAMPTON TWP, 57, 2020-06-09, True
New Jersey, Burlington, EDGEWATER PARK TWP, 100, 2020-06-09, True
New Jersey, Burlington, EVESHAM TWP, 330, 2020-06-09, True
New Jersey, Burlington, FIELDSBORO BORO, 6, 2020-06-09, True
New Jersey, Burlington, FLORENCE TWP, 89, 2020-06-09, True
New Jersey, Burlington, HAINESPORT TWP, 39, 2020-06-09, True
New Jersey, Burlington,

In [10]:


#burl_df.columns = ['City','Total Cases','Death','Date']
#burl_df.insert(0,'County','Burlington County', True)
#burl_df.insert(0,'State','New Jersey', True)
#burl_df.insert(0,'Grain','State',True)
#burl_df.head()
#headers tie to history data
#no values below the date
#burl_dg.filter(city).sort(date)(today-1)
#date datetime timestamp=> for city in cities: current = max(yesterdays, current)

In [11]:
gloc_url = 'https://services5.arcgis.com/ALQeR5k3182nooX1/arcgis/rest/services/COVID19/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&outSR=102100&resultOffset=0&resultRecordCount=25&resultType=standard&cacheHint=true'

In [12]:
gloc_req = requests.get(gloc_url)
gloc_req

<Response [200]>

In [13]:
gloc_req.content

b'{"objectIdFieldName":"FID","uniqueIdField":{"name":"FID","isSystemMaintained":true},"globalIdFieldName":"","geometryType":"esriGeometryPolygon","spatialReference":{"wkid":102100,"latestWkid":3857},"fields":[{"name":"FID","type":"esriFieldTypeOID","alias":"FID","sqlType":"sqlTypeOther","domain":null,"defaultValue":null},{"name":"MUNICIPALI","type":"esriFieldTypeString","alias":"MUNICIPALI","sqlType":"sqlTypeOther","length":50,"domain":null,"defaultValue":null},{"name":"SSN","type":"esriFieldTypeString","alias":"SSN","sqlType":"sqlTypeOther","length":8,"domain":null,"defaultValue":null},{"name":"OBJECTID","type":"esriFieldTypeInteger","alias":"OBJECTID","sqlType":"sqlTypeOther","domain":null,"defaultValue":null},{"name":"NAME","type":"esriFieldTypeString","alias":"NAME","sqlType":"sqlTypeOther","length":25,"domain":null,"defaultValue":null},{"name":"Perimeter","type":"esriFieldTypeDouble","alias":"Shape_Len","sqlType":"sqlTypeOther","domain":null,"defaultValue":null},{"name":"Area","ty

In [14]:
gloc_parsed_json = json.loads(gloc_req.content)

In [15]:
gloc_parsed_json

{'objectIdFieldName': 'FID',
 'uniqueIdField': {'name': 'FID', 'isSystemMaintained': True},
 'globalIdFieldName': '',
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'fields': [{'name': 'FID',
   'type': 'esriFieldTypeOID',
   'alias': 'FID',
   'sqlType': 'sqlTypeOther',
   'domain': None,
   'defaultValue': None},
  {'name': 'MUNICIPALI',
   'type': 'esriFieldTypeString',
   'alias': 'MUNICIPALI',
   'sqlType': 'sqlTypeOther',
   'length': 50,
   'domain': None,
   'defaultValue': None},
  {'name': 'SSN',
   'type': 'esriFieldTypeString',
   'alias': 'SSN',
   'sqlType': 'sqlTypeOther',
   'length': 8,
   'domain': None,
   'defaultValue': None},
  {'name': 'OBJECTID',
   'type': 'esriFieldTypeInteger',
   'alias': 'OBJECTID',
   'sqlType': 'sqlTypeOther',
   'domain': None,
   'defaultValue': None},
  {'name': 'NAME',
   'type': 'esriFieldTypeString',
   'alias': 'NAME',
   'sqlType': 'sqlTypeOther',
   'length': 25,
   'domain': N

In [16]:
gloc_parsed_json['features']

[{'attributes': {'FID': 1,
   'MUNICIPALI': 'Clayton Borough',
   'SSN': '0801',
   'OBJECTID': 1,
   'NAME': 'CLAYTON',
   'Perimeter': 819708.44848,
   'Area': 29144358503,
   'Acres': 4646.2667239,
   'COVID': 71,
   'Deaths': 0,
   'Recovered': 0,
   'Shape__Area': 202390568.92836,
   'Shape__Length': 68308.9007552235}},
 {'attributes': {'FID': 2,
   'MUNICIPALI': 'Deptford Township',
   'SSN': '0802',
   'OBJECTID': 2,
   'NAME': 'DEPTFORD',
   'Perimeter': 1844275.70533,
   'Area': 70541372030,
   'Acres': 11245.8824403,
   'COVID': 520,
   'Deaths': 68,
   'Recovered': 0,
   'Shape__Area': 489868679.655235,
   'Shape__Length': 153689.334656397}},
 {'attributes': {'FID': 3,
   'MUNICIPALI': 'East Greenwich Township',
   'SSN': '0803',
   'OBJECTID': 3,
   'NAME': 'E. GREENWICH',
   'Perimeter': 1063684.15437,
   'Area': 59902930261.4,
   'Acres': 9549.87537327,
   'COVID': 52,
   'Deaths': 1,
   'Recovered': 0,
   'Shape__Area': 415990907.329994,
   'Shape__Length': 88640.1688301

In [17]:
gloc_content = ''
#vim xed visualcode
#pycharm
from datetime import date
gloc_hist_df = pd.read_csv('Webscraping_Db.csv')
for gloc_city in gloc_parsed_json['features']:
    gcityname = (gloc_city['attributes']['MUNICIPALI'])
    #try to import name
    
    gloc_city_df = gloc_hist_df[gloc_hist_df['City'] == gcityname.upper()]
    gcur_data = int(gloc_city['attributes']['COVID'])
    gcurrent = max(max(gloc_city_df['Total_Cases']), gcur_data) #if Max(total)>max(confirmed) then "Call"
    gcounty = gloc_city_df['County'].values[0]
    gstate = gloc_city_df['State'].values[0]
    gdate = date.today().isoformat()
    gloc_content += f'{gstate}, {gcounty}, {gcityname}, {gcurrent}, {gdate}, {not(gcur_data ++ gcurrent)}\n'
    
print(gloc_content)
    
    #gloc_content += f'{state}, {county}, {cityname}, {current}, {date.date().isoformat()}, {not (cur_data != current)}\n'
    #cur_data is different than maximum of both current and cur_data.  
    #True means that values are correct.  False means that values are incorrect and cannot fix them
    
#print(city_df.head())
#print(gloc_content)

New Jersey, Glcouester, Clayton Borough, 71, 2020-06-10, False
New Jersey, Glcouester, Deptford Township, 520, 2020-06-10, False
New Jersey, Glcouester, East Greenwich Township, 52, 2020-06-10, False
New Jersey, Glcouester, Elk Township, 30, 2020-06-10, False
New Jersey, Glcouester, Franklin Township, 80, 2020-06-10, False
New Jersey, Glcouester, Glassboro Borough, 129, 2020-06-10, False
New Jersey, Glcouester, Greenwich Township, 23, 2020-06-10, False
New Jersey, Glcouester, Harrison Township, 58, 2020-06-10, False
New Jersey, Glcouester, Logan Township, 38, 2020-06-10, False
New Jersey, Glcouester, Mantua Township, 96, 2020-06-10, False
New Jersey, Glcouester, Monroe Township, 263, 2020-06-10, False
New Jersey, Glcouester, National Park Borough, 19, 2020-06-10, False
New Jersey, Glcouester, Newfield Borough, 8, 2020-06-10, False
New Jersey, Glcouester, Paulsboro Borough, 58, 2020-06-10, False
New Jersey, Glcouester, Pitman Borough, 42, 2020-06-10, False
New Jersey, Glcouester, South 

In [18]:
gloc_content = ''
for city in gloc_parsed_json['features']:
    (city['attributes']['MUNICIPALI'])
    gloc_content += (city['attributes']['MUNICIPALI'] +','+ str(city['attributes']['COVID']) +','+ str(city['attributes']['Deaths']) + '\n')
    
print(gloc_content)

Clayton Borough,71,0
Deptford Township,520,68
East Greenwich Township,52,1
Elk Township,29,0
Franklin Township,80,3
Glassboro Borough,129,6
Greenwich Township,23,1
Harrison Township,58,1
Logan Township,38,1
Mantua Township,96,1
Monroe Township,263,8
National Park Borough,19,2
Newfield Borough,8,1
Paulsboro Borough,58,3
Pitman Borough,42,2
South Harrison Township,80,0
Swedesboro Borough,38,1
Washington Township,345,31
Wenonah Borough,13,1
West Deptford Township,154,13
Westville Borough,37,1
Woodbury City,97,7
Woodbury Heights Borough,16,1
Woolwich Township,90,2



In [19]:
gloc_df = pd.read_csv(io.StringIO(gloc_content), header = None)
#gloc_df.head()

In [20]:
#gloc_df.columns = ['City','Total Cases','Death']
#gloc_df.insert(0,'County','Gloucester County', True)
#gloc_df.insert(0,'State','New Jersey', True)
#gloc_df.insert(0,'Grain','State',True)
#gloc_df.head()

In [21]:
camden_url = 'https://datawrapper.dwcdn.net/E7uou/'
content = requests.get(camden_url).content.decode()

In [22]:
camden_url = re.search(r'url=(.*?)"', content).group(1)
content = requests.get(camden_url).content.decode()


d = re.search(r'<meta name="description" content="Updated: ([A-Z][a-z]+\s*\d+,\s*\d+)"', content)

date = datetime.strptime(d.group(1), "%B %d, %Y")
camd_content = []
#filename = 'covid-' + '-'.join(map(str, [date.month, date.day, date.year])), + '.csv'

In [23]:
for m in re.findall(r';([A-Z\s*]+\s*(TOWNSHIP|BOROUGH))\s*<br>\s*([0-9]+);([0-9]+)', content, re.MULTILINE | re.DOTALL):
    camd_content.append([m[0], m[2], m[3], date])

print(camd_content)

[['AUDUBON BOROUGH', '53', '2', datetime.datetime(2020, 6, 10, 0, 0)], ['AUDUBON PARK BOROUGH', '6', '0', datetime.datetime(2020, 6, 10, 0, 0)], ['BARRINGTON BOROUGH', '42', '4', datetime.datetime(2020, 6, 10, 0, 0)], ['BELLMAWR BOROUGH', '118', '5', datetime.datetime(2020, 6, 10, 0, 0)], ['BERLIN BOROUGH', '52', '9', datetime.datetime(2020, 6, 10, 0, 0)], ['BERLIN TOWNSHIP', '37', '0', datetime.datetime(2020, 6, 10, 0, 0)], ['BROOKLAWN BOROUGH', '26', '0', datetime.datetime(2020, 6, 10, 0, 0)], ['CHERRY HILL TOWNSHIP', '939', '113', datetime.datetime(2020, 6, 10, 0, 0)], ['CHESILHURST BOROUGH', '27', '1', datetime.datetime(2020, 6, 10, 0, 0)], ['CLEMENTON BOROUGH', '66', '2', datetime.datetime(2020, 6, 10, 0, 0)], ['COLLINGSWOOD BOROUGH', '106', '4', datetime.datetime(2020, 6, 10, 0, 0)], ['GIBBSBORO BOROUGH', '7', '0', datetime.datetime(2020, 6, 10, 0, 0)], ['GLOUCESTER TOWNSHIP', '670', '18', datetime.datetime(2020, 6, 10, 0, 0)], ['HADDON HEIGHTS BOROUGH', '25', '3', datetime.datet

In [24]:
cam_df = pd.DataFrame(camd_content)
cam_df.head()

,0,1,2,3
0,AUDUBON BOROUGH,53,2,2020-06-10
1,AUDUBON PARK BOROUGH,6,0,2020-06-10
2,BARRINGTON BOROUGH,42,4,2020-06-10
3,BELLMAWR BOROUGH,118,5,2020-06-10
4,BERLIN BOROUGH,52,9,2020-06-10


In [25]:
cam_df.columns = ['City','Total Cases','Death', 'Date']
cam_df.insert(0,'County','Camden County', True)
cam_df.insert(0,'State','New Jersey', True)
#cam_df.insert(0,'Grain','State',True)
cam_df.head()

,State,County,City,Total Cases,Death,Date
0,New Jersey,Camden County,AUDUBON BOROUGH,53,2,2020-06-10
1,New Jersey,Camden County,AUDUBON PARK BOROUGH,6,0,2020-06-10
2,New Jersey,Camden County,BARRINGTON BOROUGH,42,4,2020-06-10
3,New Jersey,Camden County,BELLMAWR BOROUGH,118,5,2020-06-10
4,New Jersey,Camden County,BERLIN BOROUGH,52,9,2020-06-10


In [26]:
#burl_content, gloc_content, camd_content
burl_df.head()
gloc_df.head()
cam_df.head()

NameError: name 'burl_df' is not defined

In [27]:
gloc_df.head()

,0,1,2
0,Clayton Borough,71,0
1,Deptford Township,520,68
2,East Greenwich Township,52,1
3,Elk Township,29,0
4,Franklin Township,80,3


In [28]:
burl_df.head()

NameError: name 'burl_df' is not defined

In [29]:
county_df = pd.concat([burl_df,gloc_df,cam_df])

NameError: name 'burl_df' is not defined

In [30]:
county_df.head()

NameError: name 'county_df' is not defined

In [ ]:
county_df

In [ ]:
county_df.to_csv(r'3County_runon_6.03.2020_916.csv')

In [ ]:
#When are dashboards getting updated??

#NJ and PA are updated every hour
#cities are getting updated (Sunday) monday to (Thursday) Friday
#US is updated at 3 AM from the states websites
#need to change to uppercase and clean borugh/township
#Regex to Burlington County remove TWP|BORO
#Regex to Camden County and Gloucester Township|Borough
#Berlin Borough and Township are different
#Glassboro
#Franklin Township
#Fieldsboro
#Gibbsboro
#Swedesboro
#Willingboro
#Washington Township is in Gloucester

In [ ]:
#def get_output_schema():       
#  return county_df.DataFrame({
#    'Grain' : prep_int(),
#    'Supplies Subgroup Encoded' : prep_int(),
#    'Region Encoded' : prep_int(),
#    'Route To Market Encoded' : prep_int (),
#    'Opportunity Result Encoded' : prep_int (),
#    'Competitor Type Encoded' : prep_int()
#    'Supplies Group Encoded' : prep_int()
#})
#Add

In [ ]:
#!pip install tabpy